In [1]:
import pandas as pd
from sklearn.cluster import MeanShift
import numpy as np
from math import radians, cos, sin, asin, sqrt

In [2]:
data = pd.read_csv("checkins.dat", sep='|', na_values='                   ')
data.columns = data.columns.str.strip()
# x = df.replace(r'\s+', None, regex=True)
# with open('checkins.dat','r') as f:
#     next(f) # skip first row
#     df = pd.DataFrame(l.rstrip().split() for l in f)


C:\Users\Nikolay\AppData\Local\Continuum\Anaconda3\envs\py35\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# df.columns = df.columns = df.columns.str.strip()
# df['latitude'].ix[3]
df = data.dropna()

In [4]:
print(len(df))
df.head()

396634


,id,user_id,venue_id,latitude,longitude,created_at
2,984222,15824,5222,38.895112,-77.036366,2012-04-21 17:43:47
4,984234,44652,5222,33.800745,-84.410520,2012-04-21 17:43:43
8,984291,105054,5222,45.523452,-122.676207,2012-04-21 17:39:22
10,984318,2146539,5222,40.764462,-111.904565,2012-04-21 17:35:46
11,984232,93870,380645,33.448377,-112.074037,2012-04-21 17:38:18


In [5]:
sample = df[['latitude', 'longitude']][:100000]
# sample = df[['latitude', 'longitude']].sample(100000)

In [6]:
model = MeanShift(bandwidth=0.1)
clusters = model.fit_predict(sample)

In [7]:
len(clusters)

100000

In [8]:
cl = pd.Series(clusters, name='cluster')
counts = cl.value_counts()
clusters = counts.index[counts.values>15]

In [9]:
company = [
    [33.751277, -118.188740], [25.867736, -80.324116], [51.503016, -0.075479],
    [52.378894, 4.885084], [39.366487, 117.036146], [-33.868457, 151.205134]
]
company = np.array(company)

In [10]:
model.cluster_centers_

array([[  40.7177164 ,  -73.99183542],
       [  33.44943805, -112.00213969],
       [  33.44638027, -111.90188756],
       ..., 
       [  44.9592231 ,  -70.1539542 ],
       [  10.473523  ,  -84.0167423 ],
       [  40.7694969 ,  -83.8227148 ]])

In [11]:
# dist = numpy.linalg.norm(a-b)
def mindist(clcenter, company):
    dist = []
    for co in company:
        dist.append(np.linalg.norm(co - clcenter))
    return np.min(dist)
# mindist = np.vectorize(mindist)
distances = []
for cl in model.cluster_centers_:
    distances.append(mindist(cl, company))
# mindist(model.cluster_centers_[0], company)
distances = np.array(distances)
distances[:10]

array([ 16.14371999,   6.19395917,   6.29424146,  17.59857982,
         5.77204798,  13.42413138,   6.37826249,   8.885426  ,
        18.91220169,  14.46122737])

In [ ]:

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    km = 6367 * c
    return km

distances = []
for cl in model.cluster_centers_:
#     mdist = []
    for co in company:
        distances.append([haversine(cl[1], cl[0], co[1], co[0]), cl])
#     distances.append(min(mdist))
        
# mindist(model.cluster_centers_[0], company)
# distances = np.array(distances)
distances[:10]

In [ ]:
distances.sort()

In [12]:
answer = [x[1] for x in distances[:20]]
answer.sort(key=lambda x: x[0])
answer

IndexError: invalid index to scalar variable.

In [ ]:
import math
def geo_dist(shir_1, dolg_1, shir_2, dolg_2):
    # rad - радиус сферы (Земли)
    rad = 6372795

    #в радианах
    lat1 = shir_1*math.pi/180.
    lat2 = shir_2*math.pi/180.
    long1 = dolg_1*math.pi/180.
    long2 = dolg_2*math.pi/180.

    #косинусы и синусы широт и разницы долгот
    cl1 = math.cos(lat1)
    cl2 = math.cos(lat2)
    sl1 = math.sin(lat1)
    sl2 = math.sin(lat2)
    delta = long2 - long1
    cdelta = math.cos(delta)
    sdelta = math.sin(delta)

    #вычисления длины большого круга
    y = math.sqrt(math.pow(cl2*sdelta,2)+math.pow(cl1*sl2-sl1*cl2*cdelta,2))
    x = sl1*sl2+cl1*cl2*cdelta
    ad = math.atan2(y,x)
    dist = ad*rad
    
    return dist

distances = []
for cl in model.cluster_centers_:
    mdist = []
    for co in company:
        mdist.append(geo_dist(cl[0], cl[1], co[0], co[1]))
    distances.append(min(mdist))
        
# mindist(model.cluster_centers_[0], company)
distances = np.array(distances)
distances[:10]

In [15]:
# np.linalg.norm(company[2] - model.cluster_centers_[0])
ind = np.argsort(distances)[:20]
ind
distances[411]

0.007834758163107856

In [16]:
coord = model.cluster_centers_[ind]
coord = np.round(coord, decimals=2)
coord

array([[ -3.38600000e+01,   1.51200000e+02],
       [  5.23700000e+01,   4.89000000e+00],
       [  2.58500000e+01,  -8.03200000e+01],
       [  5.15000000e+01,  -1.30000000e-01],
       [  3.38100000e+01,  -1.18150000e+02],
       [  2.57900000e+01,  -8.02200000e+01],
       [ -3.40000000e+01,   1.51130000e+02],
       [  2.57100000e+01,  -8.02800000e+01],
       [  2.60100000e+01,  -8.02000000e+01],
       [ -3.39500000e+01,   1.51030000e+02],
       [  3.38900000e+01,  -1.18050000e+02],
       [  3.38700000e+01,  -1.18360000e+02],
       [  3.39700000e+01,  -1.18170000e+02],
       [  5.14300000e+01,  -3.00000000e-01],
       [  5.23900000e+01,   4.63000000e+00],
       [  5.15700000e+01,   1.80000000e-01],
       [  2.61400000e+01,  -8.03300000e+01],
       [  5.22600000e+01,   4.63000000e+00],
       [  5.15100000e+01,  -3.70000000e-01],
       [  3.39800000e+01,  -1.18010000e+02]])

In [17]:
# np.sort(coord, axis=0)
ind = np.lexsort((coord[:,1],coord[:,0])) 
coord = coord[ind]
coord

array([[ -3.40000000e+01,   1.51130000e+02],
       [ -3.39500000e+01,   1.51030000e+02],
       [ -3.38600000e+01,   1.51200000e+02],
       [  2.57100000e+01,  -8.02800000e+01],
       [  2.57900000e+01,  -8.02200000e+01],
       [  2.58500000e+01,  -8.03200000e+01],
       [  2.60100000e+01,  -8.02000000e+01],
       [  2.61400000e+01,  -8.03300000e+01],
       [  3.38100000e+01,  -1.18150000e+02],
       [  3.38700000e+01,  -1.18360000e+02],
       [  3.38900000e+01,  -1.18050000e+02],
       [  3.39700000e+01,  -1.18170000e+02],
       [  3.39800000e+01,  -1.18010000e+02],
       [  5.14300000e+01,  -3.00000000e-01],
       [  5.15000000e+01,  -1.30000000e-01],
       [  5.15100000e+01,  -3.70000000e-01],
       [  5.15700000e+01,   1.80000000e-01],
       [  5.22600000e+01,   4.63000000e+00],
       [  5.23700000e+01,   4.89000000e+00],
       [  5.23900000e+01,   4.63000000e+00]])

In [ ]:
coord.flatten()

In [ ]:
# def writeans(ans, file):
#     with open(file, 'w') as f:
#         f.write(str(ans))
def writeans(list_pc1):
    with open("answer222.txt", "w") as fout:
        fout.write(" ".join([str(num) for num in list_pc1]))        

In [ ]:
writeans(coord.flatten())

In [ ]:
coord